# Network analysis notebook

This contains microbiome network analysis using Sparse Correlation Network Analysis for Compositional Data (SCNIC). This notebook does not use a QIIME environment. Instead, it uses an environment for SCNIC.

I exported the `100_filtered-table.qza` artifact using 
> qiime tools export \
--input-path 100_filtered-table.qza \
--output-path 100_filtered-table.biom

cleanup with

> mv 100_filtered-table.biom/feature-table.biom . \
rm -r 100_filtered-table.biom \
mv feature-table.biom 100_filtered-table.biom

In [19]:
import pandas as pd
import biom
from SCNIC import within_correls, module_analysis

In [ ]:
within_correls.within_correls(input_loc="100_filtered-table.biom",
                              output_loc="SCNIC_out")